In [4]:
import pickle

import gymnasium as gym
import numpy as np
from helper import initialize_grids, initialize_q_table, initialize_state_dict, initialize_random_start, \
    epsilon_greedy_policy, get_closest_in_grid, plot_rewards, plot_steps, evaluate
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


def train(q, rand_init=True):
    # Initialize variables to track rewards
    reward_list = []
    avg_reward_list = []
    total_steps = []
    file = open('trace/trace_train.txt', 'w')

    for episode in range(n_training_episodes):
        epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)
        state = env.reset()
        if rand_init:
            state = initialize_random_start(grid_x, grid_v)
        steps = 0
        tot_reward, reward = 0, 0
        terminated = False

        for step in range(max_steps):
            # Choose the action At using epsilon greedy policy
            action = epsilon_greedy_policy(q, state, epsilon, grid_x, grid_v, state_to_qtable, env)

            file.write(f'{state[0]},{state[1]},{action}\n')

            new_state, reward, terminated, truncated, info = env.step(action)

            s = state_to_qtable[get_closest_in_grid(state, grid_x, grid_v)]
            ns = state_to_qtable[get_closest_in_grid(new_state, grid_x, grid_v)]

            # Calculate current learning rate
            lr = max(min_lr, initial_lr * np.exp(-k * episode))

            # Update Q table
            q[s][action] = q[s][action] + lr * (reward + gamma * np.max(q[ns]) - q[s][action])

            steps += 1

            # If done, finish the episode
            if terminated:  # or truncated:
                total_steps.append(steps)
                break

            # Our state is the new state
            state = new_state

            # Update total reward
            tot_reward += reward

            # Track rewards
            reward_list.append(tot_reward)

        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(reward_list)
            avg_reward_list.append(avg_reward)
            reward_list = []
            print(f"episode: {episode}\t average reward: {avg_reward}\t avg steps: {np.mean(total_steps[-10:])}"
                  f"\t lr: {lr}\t epsilon: {epsilon}")

    return q, avg_reward_list, total_steps

In [6]:
env = gym.make("MountainCar-v0")

# initialize size of state and action space
state_space = 20 * 20
action_space = env.action_space.n

observation, info = env.reset(seed=42)

# Training parameters
n_training_episodes = 50000  # Total training episodes
initial_lr = 0.2  # Learning rate       # old: 1
k = 0.0005  # lr decay                    # old: 0.005
min_lr = 0.0005


# Environment parameters
max_steps = 10000  # Max steps per episode
gamma = 0.99  # Discounting rate
eval_seed = []  # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1#0.95   # Exploration probability at start                     # old: 1
min_epsilon = 0.05  # Minimum exploration probability
decay_rate = 0.001  # Exponential decay rate for exploration prob           # old: 0.002


grid_x, grid_v = initialize_grids()
state_to_qtable = initialize_state_dict()

# Training
q_car = initialize_q_table(state_space, action_space)
q_car, avg_rewards, total_steps = train(q_car)


#
# Plot Q values for random init training
actions = np.max(q_car, axis=1)
actions = actions.reshape((20, 20))
plt.figure(figsize=(16, 12))
ax = plt.subplot(111)
ax = sns.heatmap(actions, annot=True)
plt.ylim(0, 20)
plt.xlabel("Position", fontsize=20)
plt.ylabel("Velocity", fontsize=20)
plt.title("Q values for optimal action - random init w/ lr decay", fontdict={'fontsize': 25})
# plt.savefig('plots/v_values_rand_decay')
with open('plots/q_values.pkl','wb') as fid:
    pickle.dump(ax, fid)
plt.savefig('plots/q_values_rand_decay.png')
plt.close()

plot_steps(total_steps, 'rand_decay')
plot_rewards(avg_rewards, 'rand_decay')

# # Save Q table
np.savetxt('data/q_rand_decay.txt', q_car)
np.array(avg_rewards)
np.savetxt('data/avg_rewards_rand_decay.txt', avg_rewards)
np.array(total_steps)
np.savetxt('data/total_steps_rand_decay.txt', total_steps)

env.close()

C:\Users\lenna\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\lenna\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


episode: 9	 average reward: -5000.5	 avg steps: nan	 lr: 0.19910202196591412	 epsilon: 0.9914883598342394
episode: 19	 average reward: -4557.308114536639	 avg steps: 6863.0	 lr: 0.1981089964885801	 epsilon: 0.9821203941306657
episode: 29	 average reward: -4365.029278453155	 avg steps: 5977.0	 lr: 0.1971209237464765	 epsilon: 0.9728456412432746
episode: 39	 average reward: -3751.307521902795	 avg steps: 5594.1	 lr: 0.19613777903773325	 epsilon: 0.9636631736890484
episode: 49	 average reward: -4739.215335731415	 avg steps: 4405.7	 lr: 0.19515953778368147	 epsilon: 0.9545720732135796
episode: 59	 average reward: -3868.677663332836	 avg steps: 5098.3	 lr: 0.19418617552823886	 epsilon: 0.9455714306992448
episode: 69	 average reward: -4235.458823688006	 avg steps: 5245.5	 lr: 0.19321766793729833	 epsilon: 0.9366603460742918
episode: 79	 average reward: -3988.238947022798	 avg steps: 5752.9	 lr: 0.19225399079811967	 epsilon: 0.9278379282228324
episode: 89	 average reward: -4052.8162255645493	

episode: 689	 average reward: -263.78660915999177	 avg steps: 487.9	 lr: 0.14171491043736692	 epsilon: 0.5269740012064343
episode: 699	 average reward: -292.64963346269946	 avg steps: 464.8	 lr: 0.14100810437285335	 epsilon: 0.5222280305971058
episode: 709	 average reward: -247.85723556132382	 avg steps: 463.3	 lr: 0.14030482351829335	 epsilon: 0.5175292831843616
episode: 719	 average reward: -329.49413833528723	 avg steps: 512.8	 lr: 0.1396050502916288	 epsilon: 0.512877289089545
episode: 729	 average reward: -373.0449685534591	 avg steps: 637.0	 lr: 0.13890876719849268	 epsilon: 0.5082715831093696
episode: 739	 average reward: -224.2089150134705	 avg steps: 409.3	 lr: 0.13821595683177135	 epsilon: 0.5037117046693995
episode: 749	 average reward: -234.81237911025144	 avg steps: 414.6	 lr: 0.13752660187116958	 epsilon: 0.49919719777799054
episode: 759	 average reward: -233.53235225414622	 avg steps: 429.1	 lr: 0.13684068508277744	 epsilon: 0.4947276109806917
episode: 769	 average rewar

episode: 1359	 average reward: -140.35737586081913	 avg steps: 276.9	 lr: 0.10137407283989047	 epsilon: 0.2940716877985012
episode: 1369	 average reward: -307.97703788748566	 avg steps: 436.5	 lr: 0.10086846754227896	 epsilon: 0.2916431339277849
episode: 1379	 average reward: -172.42777413000655	 avg steps: 305.6	 lr: 0.10036538396160961	 epsilon: 0.28923874457183135
episode: 1389	 average reward: -121.45701943844493	 avg steps: 232.5	 lr: 0.09986480952076665	 epsilon: 0.28685827928970137
episode: 1399	 average reward: -125.73900414937759	 avg steps: 242.0	 lr: 0.099366731705363	 epsilon: 0.2845015000328828
episode: 1409	 average reward: -115.49582947173309	 avg steps: 216.8	 lr: 0.09887113806342734	 epsilon: 0.2821681711214862
episode: 1419	 average reward: -141.91647940074907	 avg steps: 268.0	 lr: 0.09837801620509283	 epsilon: 0.27985805922067575
episode: 1429	 average reward: -191.1558516801854	 avg steps: 346.2	 lr: 0.0978873538022873	 epsilon: 0.27757093331733645
episode: 1439	 a

episode: 2029	 average reward: -120.71721132897603	 avg steps: 230.5	 lr: 0.07251673527105228	 epsilon: 0.17489357624133203
episode: 2039	 average reward: -127.6517269736842	 avg steps: 244.2	 lr: 0.07215505654500913	 epsilon: 0.1736508643940698
episode: 2049	 average reward: -90.45444319460067	 avg steps: 178.8	 lr: 0.07179518169913772	 epsilon: 0.17242051773628975
episode: 2059	 average reward: -113.96134606639382	 avg steps: 220.9	 lr: 0.07143710173654812	 epsilon: 0.17120241323230073
episode: 2069	 average reward: -107.57509505703422	 avg steps: 211.4	 lr: 0.07108080770522261	 epsilon: 0.16999642907063733
episode: 2079	 average reward: -107.24735322425408	 avg steps: 208.8	 lr: 0.07072629069779186	 epsilon: 0.16880244465187827
episode: 2089	 average reward: -113.23722789871168	 avg steps: 226.1	 lr: 0.07037354185131224	 epsilon: 0.16762034057658687
episode: 2099	 average reward: -99.47013388259526	 avg steps: 195.2	 lr: 0.07002255234704417	 epsilon: 0.16644999863337032
episode: 210

...

episode: 47149	 average reward: -78.94042280589366	 avg steps: 157.1	 lr: 0.0005	 epsilon: 0.05
episode: 47159	 average reward: -79.3879365079365	 avg steps: 158.5	 lr: 0.0005	 epsilon: 0.05
episode: 47169	 average reward: -78.03102779573368	 avg steps: 155.7	 lr: 0.0005	 epsilon: 0.05
episode: 47179	 average reward: -75.71124417831005	 avg steps: 151.3	 lr: 0.0005	 epsilon: 0.05
episode: 47189	 average reward: -75.80735785953178	 avg steps: 150.5	 lr: 0.0005	 epsilon: 0.05
episode: 47199	 average reward: -72.85594405594405	 avg steps: 144.0	 lr: 0.0005	 epsilon: 0.05
episode: 47209	 average reward: -73.39623955431755	 avg steps: 144.6	 lr: 0.0005	 epsilon: 0.05
episode: 47219	 average reward: -73.29647546648238	 avg steps: 145.7	 lr: 0.0005	 epsilon: 0.05
episode: 47229	 average reward: -72.88440111420613	 avg steps: 144.6	 lr: 0.0005	 epsilon: 0.05
episode: 47239	 average reward: -70.27545787545787	 avg steps: 137.5	 lr: 0.0005	 epsilon: 0.05
episode: 47249	 average reward: -75.63848

episode: 48009	 average reward: -77.15862524785194	 avg steps: 152.3	 lr: 0.0005	 epsilon: 0.05
episode: 48019	 average reward: -78.41365979381443	 avg steps: 156.2	 lr: 0.0005	 epsilon: 0.05
episode: 48029	 average reward: -72.62991573033707	 avg steps: 143.4	 lr: 0.0005	 epsilon: 0.05
episode: 48039	 average reward: -74.87759336099585	 avg steps: 145.6	 lr: 0.0005	 epsilon: 0.05
episode: 48049	 average reward: -75.14844804318489	 avg steps: 149.2	 lr: 0.0005	 epsilon: 0.05
episode: 48059	 average reward: -75.87959866220736	 avg steps: 150.5	 lr: 0.0005	 epsilon: 0.05
episode: 48069	 average reward: -71.65103056147832	 avg steps: 141.7	 lr: 0.0005	 epsilon: 0.05
episode: 48079	 average reward: -75.28881355932204	 avg steps: 148.5	 lr: 0.0005	 epsilon: 0.05
episode: 48089	 average reward: -75.40080160320642	 avg steps: 150.7	 lr: 0.0005	 epsilon: 0.05
episode: 48099	 average reward: -75.00402955003358	 avg steps: 149.9	 lr: 0.0005	 epsilon: 0.05
episode: 48109	 average reward: -76.7872

episode: 48879	 average reward: -74.34010840108401	 avg steps: 148.6	 lr: 0.0005	 epsilon: 0.05
episode: 48889	 average reward: -73.72131147540983	 avg steps: 147.4	 lr: 0.0005	 epsilon: 0.05
episode: 48899	 average reward: -74.18138586956522	 avg steps: 148.2	 lr: 0.0005	 epsilon: 0.05
episode: 48909	 average reward: -73.9550408719346	 avg steps: 147.8	 lr: 0.0005	 epsilon: 0.05
episode: 48919	 average reward: -74.16700610997964	 avg steps: 148.3	 lr: 0.0005	 epsilon: 0.05
episode: 48929	 average reward: -73.87593728698023	 avg steps: 147.7	 lr: 0.0005	 epsilon: 0.05
episode: 48939	 average reward: -74.27137042062415	 avg steps: 148.4	 lr: 0.0005	 epsilon: 0.05
episode: 48949	 average reward: -72.4394993045897	 avg steps: 144.8	 lr: 0.0005	 epsilon: 0.05
episode: 48959	 average reward: -70.71632216678546	 avg steps: 141.3	 lr: 0.0005	 epsilon: 0.05
episode: 48969	 average reward: -73.20853406744666	 avg steps: 146.3	 lr: 0.0005	 epsilon: 0.05
episode: 48979	 average reward: -87.084649

episode: 49739	 average reward: -75.399064171123	 avg steps: 150.6	 lr: 0.0005	 epsilon: 0.05
episode: 49749	 average reward: -72.06991525423729	 avg steps: 142.6	 lr: 0.0005	 epsilon: 0.05
episode: 49759	 average reward: -74.15953835709436	 avg steps: 148.3	 lr: 0.0005	 epsilon: 0.05
episode: 49769	 average reward: -70.45896877269426	 avg steps: 138.7	 lr: 0.0005	 epsilon: 0.05
episode: 49779	 average reward: -74.76429567642957	 avg steps: 144.4	 lr: 0.0005	 epsilon: 0.05
episode: 49789	 average reward: -69.97520058351569	 avg steps: 138.1	 lr: 0.0005	 epsilon: 0.05
episode: 49799	 average reward: -75.50803212851406	 avg steps: 150.4	 lr: 0.0005	 epsilon: 0.05
episode: 49809	 average reward: -73.49072164948454	 avg steps: 146.5	 lr: 0.0005	 epsilon: 0.05
episode: 49819	 average reward: -70.60057887120115	 avg steps: 139.2	 lr: 0.0005	 epsilon: 0.05
episode: 49829	 average reward: -72.4179523141655	 avg steps: 143.6	 lr: 0.0005	 epsilon: 0.05
episode: 49839	 average reward: -71.6281050

ImportError: DLL load failed while importing _backend_agg: The specified module could not be found.

In [7]:
from helper import greedy_policy, initialize_grids, initialize_state_dict, get_closest_in_grid
import matplotlib.image as img


# Evaluation Parameters
seed = []
n_eval_episodes = 1
max_steps = 10000

# load q table
q = np.loadtxt('data/q_rand_decay.txt')

# initialize discretization
grid_x, grid_v = initialize_grids()
state_to_qtable = initialize_state_dict()

# trace
file = open('trace/trace_eval.txt', 'w')

env = gym.make('MountainCar-v0')

for episode in range(n_eval_episodes):
    if seed:
        state = env.reset(seed=seed[episode])
    else:
        state = env.reset()[0]
    step = 0
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
        action = greedy_policy(q, state, grid_x, grid_v, state_to_qtable)

        # write current state and action taken to trace
        file.write(f'{state[0]},{state[1]},{action}\n')

        new_state, reward, terminated, truncated, info = env.step(action)
        total_rewards_ep += reward

        if terminated:  # or truncated:
            break
        state = new_state


In [10]:
trace = []
actions = []

# load trace and actions into list
with open('trace/trace_eval.txt') as f:
    for line in f.readlines():
        x, v, a = line.split(',')
        trace.append((float(x), float(v)))
        actions.append(int(a))

# discretize trace
trace_discrete = []
grid_x, grid_v = initialize_grids()
state_to_q = initialize_state_dict()
for trac in trace:
    # state = state_to_q[get_closest_in_grid(np.asarray(trac), grid_x, grid_v)]
    x, v = get_closest_in_grid(np.asarray(trac), grid_x, grid_v)
    state = [grid_x.index(x)+0.5, grid_v.index(v)+0.5]
    trace_discrete.append(state)

# load v values plot
plot = img.imread('plots/q_values_rand_decay.png')

# plot trace onto plot
fig, ax = plt.subplots()
# ax.imshow(plot, extent=[0, 20, 0, 20])
with open('plots/q_values.pkl', 'rb') as fid:
    ax = pickle.load(fid)
ax.plot(*zip(*trace_discrete[::1]))

# plot starting point
sns.lineplot(x=[trace_discrete[0][0]], y=[trace_discrete[0][1]], marker='o', markersize=25, markeredgecolor='green', color='green', markeredgewidth=2)
# plot finish point
sns.lineplot(x=[trace_discrete[-1][0]+1], y=[trace_discrete[-1][1]], marker='o', markersize=25, markeredgecolor='red', color='red', markeredgewidth=2)

ax.set_xticklabels(grid_x)
ax.set_yticklabels(grid_v)
plt.savefig('trace/trace.png')
plt.close()


ImportError: DLL load failed while importing _backend_agg: The specified module could not be found.